<a href="https://colab.research.google.com/github/CarolGitonga/Natural-Language-Processing/blob/main/Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Change the runtime type on Colab to GPU by following these steps:

On the menu bar, click on Runtime.
Click change runtime type.
In the Hardware accelerator radio options, select T4 GPU.
Click save.

In [1]:
! pip install transformers
! pip install accelerate
! pip install optimum
! pip install auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from transformers import pipeline
import accelerate

Ignore the warrnings to keep the cell outputs nice and clean

In [3]:
import warnings
warnings.filterwarnings("ignore")

def warn(*args, **kwargs):
    pass
warnings.warn = warn

Create LLaMA-2 Pipeline

In [4]:
model = "TheBloke/Llama-2-13B-chat-GPTQ"
llama_pipe = pipeline("text-generation", model=model, device_map="auto")

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at TheBloke/Llama-2-13B-chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


A Helper Function

In [5]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    This function takes a prompt and passes it to the model e.g. LLaMA and returnss the response from the model

    Parameters:
    @param prompt (str): The input text prompt to generate a response for.
    @param max_length (int): The maximum length, in tokens, of the generated response.
    @param pipe (callable): The model's pipeline function used for generation.
    @param **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

First prompt: Capital of Kenya. Begin with a very simple prompt and pass it to the LLaMA-2 model. The desired outcome is that the model responds to us with only the name of the capital of Kenya, which is Nairobi, with nothing else in the response

In [9]:
prompt = "What is the capital of Kenya?"

print(generate(prompt))

Answer: The capital of Kenya is Nairobi.


The model responds back with additional helpful information about the city of Nairobi. We are not interested in this additional information. We want the name of the city and no additional context, so let's craft a prompt that is more specific.

In [10]:
prompt = "What is the capital of Kenya? Only answer this question and do so in as few a words as possible."

print(generate(prompt))

Answer: Nairobi


That was a better response, but we still get a leading Answer: in the reponse. We can prevent this behaviour by providing the model with the cue Answer:. Doing this can prevent the model from providing the text itself.

In [13]:
prompt = "What is the capital of Kenya? Only answer this question and do so in as few a words as possible. Answer: "

print(generate(prompt))

Nairobi.


Second prompt: Vowels in 'Nairobi'

In [14]:
prompt = "Tell me the vowels in the capital of Kenya."

print(generate(prompt))

Answer: The vowels in the capital of Kenya are "e" and "a".


When LLMs are faced with the need to reason in a way that requires multiple steps, it is often helpful to craft a prompt instructing the model to perform multiple intermediary steps, like asking it to show its working. This technique is often described as giving the model "time to think".

In [16]:
prompt = "Tell me the capital of Kenya, and then tell me all the vowels in it."

print(generate(prompt))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


I'll start: The capital of Kenya is Nairobi.

Now, can you tell me all the vowels in Nairobi?


We can see that giving the model time to think made a big difference. Armed with this new technique, let's ask the model to do something more complicated - tell the vowels in the name of the capital of Kenya in reverse alphabetical order.

In [17]:
prompt = "Tell me the vowels in the capital of Kenya in reverse alphabetical order?"

print(generate(prompt))

I'm not sure what you're asking. The capital of Kenya is Nairobi, and there are no vowels in the name Nairobi.


Obviously, we did not give the model time to think. Let's ask it to break the task down to intermediate steps and show its work.

In [21]:
prompt = "Tell me the capital of Kenya, and then tell me all the vowels in it, then tell me the vowels in reverse-alphabetical order."

print(generate(prompt))

I'm not sure if this is a trick question, but I'll do my best to answer it!

The capital of Kenya is Nairobi.

The vowels in Nairobi are:

* A
* E
* I

In reverse alphabetical order, the vowels in Nairobi are:

* I
* E
* A


Key concepts:

Precise: Being as specific and explicit as necessary to guide the response of an LLM.

Cue: A phrase to conclude your prompt to guides its response, and often to prevent it from including the cue itself in its response.

"Time to think": A quality in prompts that supports LLM responses (often requiring calculation) by asking for the model to take multiple steps and show its work.
